# Lab Assignment One: Exploring Table Data

#### Everett Cienkus, Blake Miller, Colin Weil

## 1. Business Understanding
This dataset contains data for all NBA games from the 2004-2005 season until March of the 2021-2022 season.From the description of the dataset, the data was collected from the NBA stats website.

This dataset can be useful for various parties, including NBA teams, coaches, players, sports bettors, or anyone looking to better understand basketball statistics. In our exploration we will be specifically looking at the application of this data for an NBA team General Manager.

According to www.investopedia.com, of the 30 NBA teams "a team is worth on average $2.12 billion", and "total revenue across the organization reached 8.76 billion in the 2018-2019 season." This makes the role of a GM extremely important, as they must manage a multi-billion dollar business.

Through analyzing thousands of NBA game data from this dataset, a GM could make better informed decisions on what statistics are most impactful on the game, and use that information to draft players which have the strengths that they need to build their team.


## 2. Data Understanding

### 2.1 Data Description

In [70]:
import pandas as pd
import numpy as np

print('Pandas:', pd.__version__)
print('Numpy:',np.__version__)
df = pd.read_csv('NBA_datasets/games.csv') # read in the csv file
#Split stats into home and away
home_df = df[["GAME_DATE_EST","PTS_home","FG_PCT_home","FT_PCT_home","FG3_PCT_home","AST_home","REB_home", "HOME_TEAM_WINS"]]
away_df = df[["GAME_DATE_EST","PTS_away","FG_PCT_away","FT_PCT_away","FG3_PCT_away","AST_away","REB_away", "HOME_TEAM_WINS"]]

#Convert HOME_TEAM_WINS the to appropriate value based on home or away team
home_df=home_df.rename(columns={'HOME_TEAM_WINS':'WIN'})
away_df=away_df.rename(columns={'HOME_TEAM_WINS':'WIN'})
away_df["WIN"]=np.logical_not(away_df["WIN"]).astype(int)

home_df.columns = ['GAME_DATE_EST', 'PTS', 'FG_PCT', 'FT_PCT', 'FG3_PCT', 'AST', 'REB', 'WIN']
home_df["HOME"]= 1
away_df.columns = ['GAME_DATE_EST', 'PTS', 'FG_PCT', 'FT_PCT', 'FG3_PCT', 'AST', 'REB', 'WIN']
away_df["HOME"]= 0
df = pd.concat([home_df, away_df])

#display
display(df.head())

Pandas: 1.4.2
Numpy: 1.21.5


,GAME_DATE_EST,PTS,FG_PCT,FT_PCT,FG3_PCT,AST,REB,WIN,HOME
0,2022-03-12,104.0,0.398,0.760,0.333,23.0,53.0,0,1
1,2022-03-12,101.0,0.443,0.933,0.429,20.0,46.0,1,1
2,2022-03-12,108.0,0.412,0.813,0.324,28.0,52.0,0,1
3,2022-03-12,122.0,0.484,0.933,0.400,33.0,55.0,1,1
4,2022-03-12,115.0,0.551,0.750,0.407,32.0,39.0,0,1


(NEED TO DO) Need to describe the changes made

In [71]:
# find the data type
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51592 entries, 0 to 25795
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   GAME_DATE_EST  51592 non-null  object 
 1   PTS            51394 non-null  float64
 2   FG_PCT         51394 non-null  float64
 3   FT_PCT         51394 non-null  float64
 4   FG3_PCT        51394 non-null  float64
 5   AST            51394 non-null  float64
 6   REB            51394 non-null  float64
 7   WIN            51592 non-null  int64  
 8   HOME           51592 non-null  int64  
dtypes: float64(6), int64(2), object(1)
memory usage: 3.9+ MB
None


(NEED TO DO) Describe the data

In [74]:
# find the data summary
display(df.describe())

,PTS,FG_PCT,FT_PCT,FG3_PCT,AST,REB,WIN,HOME
count,51394.000000,51394.000000,51394.000000,51394.000000,51394.000000,51394.000000,51592.000000,51592.000000
mean,101.700082,0.454789,0.758893,0.352655,22.070339,42.715473,0.500000,0.500000
std,13.333345,0.056352,0.102066,0.111117,5.202080,6.604647,0.500005,0.500005
min,33.000000,0.244000,0.143000,0.000000,4.000000,15.000000,0.000000,0.000000
25%,93.000000,0.417000,0.696000,0.281000,18.000000,38.000000,0.000000,0.000000
50%,101.000000,0.453000,0.765000,0.353000,22.000000,42.000000,0.500000,0.500000
75%,111.000000,0.494000,0.829000,0.423000,25.000000,47.000000,1.000000,1.000000
max,168.000000,0.687000,1.000000,1.000000,50.000000,81.000000,1.000000,1.000000


In [81]:
# create a data description table
data_des = pd.DataFrame()

data_des['Features'] = df.columns

data_des['Description'] = ['date the game occured', 'number of points scored', 'percentage of all shots made',
                          'percentage of free throws made', 'percentage of three point shots made',
                          'number of assists', 'number of rebounds',
                          'whether the game was won', 'whether the team was home or away']

data_des['Scales'] = ['ordinal'] * 2 + ['ratio'] * 3 + ['ordinal'] * 2 + ['norminal'] * 2

data_des['Discrete\Continuous'] = ['discrete'] + ['continuous'] * 6  + ['discrete'] * 2

data_des['Range'] = ['Placeholder','33 - 168','.244 - .687','.143 - 1','0 - 1','4 - 50','15 - 81','0: No; 1: Yes','0: No; 1: Yes']

data_des

,Features,Description,Scales,Discrete\Continuous,Range
0,GAME_DATE_EST,date the game occured,ordinal,discrete,Placeholder
1,PTS,number of points scored,ordinal,continuous,33 - 168
2,FG_PCT,percentage of all shots made,ratio,continuous,.244 - .687
3,FT_PCT,percentage of free throws made,ratio,continuous,.143 - 1
4,FG3_PCT,percentage of three point shots made,ratio,continuous,0 - 1
5,AST,number of assists,ordinal,continuous,4 - 50
6,REB,number of rebounds,ordinal,continuous,15 - 81
7,WIN,whether the game was won,norminal,discrete,0: No; 1: Yes
8,HOME,whether the team was home or away,norminal,discrete,0: No; 1: Yes


(NEED TO DO) Need to describe the changes made

In [82]:
# find the duplicate instances
idx = df.duplicated()

# find the number of duplicate (not first show)
len(df[idx])


190

(NEED TO DO) Describe the data

In [26]:
# find the data summary
display(home_df.describe())
display(away_df.describe())

### 2.2 Data Quality

In [28]:
# Code goes here

## 3. Data Visualization

### 3.1 Data Exploration

In [29]:
# Code goes here

### 3.2 Data Relationship Exploration

In [ ]:
# Code goes here

## 4. Dimensionality Reduction

In [ ]:
# Code goes here

In [19]:
# Code goes here